In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from typing import Tuple
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv


c:\Users\Sayantan Datta\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Loading the model and creating the predict function

In [3]:
def load_model():
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('./emotion_detection_model')
    return tokenizer, model

def predict_top_emotions(text: str, top_k=3) -> Tuple[list, list]:
    """
    Predicts top K emotions from text using the loaded model
    Returns tuple of (emotions, confidences)
    """
    tokenizer, model = load_model()
    # Prepare the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get top K predictions
    top_k_probs, top_k_indices = torch.topk(predictions, top_k, dim=1)
    
    # Map the predictions to emotion labels
    emotion_map = {
        0: "sadness",
        1: "joy",
        2: "love",
        3: "anger",
        4: "fear",
        5: "surprise"
    }
    
    top_k_emotions = [emotion_map[idx.item()] for idx in top_k_indices[0]]
    top_k_confidences = top_k_probs[0].tolist()
    
    return top_k_emotions, top_k_confidences
 

# Translation function

In [4]:
def translate_to_english(text: str) -> str:
    """
    Translates the input text to English using GPT-4
    """
    translate_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")
    parser = StrOutputParser()

    chain = translate_model | parser

    template = """
    You are a helpful AI assistant that translates text to English.
    Please translate the following text to English if it's not already in English.
    If the text is already in English, return it unchanged.

    text: {text}
    """

    prompt = ChatPromptTemplate.from_template(template)

    chain = prompt | translate_model | parser

    output = chain.invoke({
        "text": text
    })

    return output

# Transformation function

In [11]:
def transform_emotion(text: str, emotion: str) -> str:
    transform_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4")
    parser = StrOutputParser()

    chain = transform_model | parser

    template = """
    You are a helpful AI assistant that transforms a given text to another tone/emotion as specified by the user. The user
    will specify the emotion like "more sad" or "less happy" etc. Also, if it is in another language, preserve it while transforming (no need to mention this in your response).

    text: {text}

    emotion: {emotion}
    """

    prompt = ChatPromptTemplate.from_template(template)

    chain = prompt | transform_model | parser

    output = chain.invoke({
        "text": text,
        "emotion": emotion
    })

    return output



In [8]:
text1 = "I can’t believe how rude she was today. I was just asking a simple question and she snapped at me for no reason. It’s really frustrating when people are so disrespectful." #english (anger)
text2 = "রাতে একা বাড়িতে বসে থাকলে আমি ভয় পাই। প্রতিটি ছোট শব্দে আমার হৃদয় দ্রুত ধড়ফড় করতে থাকে, আর আমি ভাবতে থাকি, কোথাও কি কেউ আমার কাছে আসছে না? অন্ধকারে সব কিছু অদ্ভুত লাগে।" #bengali (fear)
text3 = "ਅੱਜ ਮੈਨੂੰ ਬਹੁਤ ਖੁਸ਼ੀ ਹੋ ਰਹੀ ਹੈ ਕਿਉਂਕਿ ਮੈਂ ਆਪਣੇ ਕਠੋਰ ਮਿਹਨਤ ਦਾ ਫਲ ਪ੍ਰਾਪਤ ਕੀਤਾ ਹੈ। ਮੈਨੂੰ ਲੱਗਦਾ ਹੈ ਹੁਣ ਮੇਰੇ ਸਾਰੇ ਸੁਪਨੇ ਸੱਚੇ ਹੋ ਸਕਦੇ ਹਨ ਅਤੇ ਮੈਂ ਬਹੁਤ ਉਤਸ਼ਾਹਿਤ ਮਹਿਸੂਸ ਕਰ ਰਿਹਾ ਹਾਂ।" #punjabi (joy)
text4 = "आज मुझे बहुत उदास महसूस हो रहा है। ऐसा लग रहा है जैसे सब कुछ गलत हो रहा है और मुझे नहीं पता कि इससे कैसे बाहर आऊं। कुछ अच्छा करने की कोशिश करता हूँ, लेकिन फिर भी दिल भारी है।" #hindi (sadness)

In [9]:
text = translate_to_english(text1)
print(predict_top_emotions(text))

(['anger', 'sadness', 'joy'], [0.9976836442947388, 0.001082075061276555, 0.00047183403512462974])


In [13]:
target_emotion = "less angry"

In [14]:
print(transform_emotion(text, target_emotion))

"I'm rather taken aback by her behavior today. I was merely asking a straightforward question and she responded somewhat hastily, without any apparent cause. It's somewhat disappointing when people lack basic courtesy."
